<a href="https://colab.research.google.com/github/Sourav56/Computer-Vision/blob/master/detector_webcam_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import numpy as np
from PIL import Image
import io
import cv2

In [ ]:
import cv2
from pnslib import utils
import matplotlib.pyplot as plt


In [ ]:
# install pnslib for updated face detector using harr cascade
!pip install git+git://github.com/PnS2019/pnslib.git
!wget https://pns2019.github.io/images/Lenna.png

In [ ]:
import os
import zipfile
import pandas as pd
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
from os import listdir
import cv2
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving my_h5_model.h5 to my_h5_model (1).h5


In [ ]:
face_mask = tf.keras.models.load_model('my_h5_model (1).h5')

In [ ]:
face_mask.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 7, 7, 100)         1152100   
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 3, 100)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 3, 100)         90100     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 100)         0         
_________________________________________________________________
flatten (Flatten)            (None, 100)               0         
_________________________________________________________________
dropout (Dropout)            (None, 100)               0

In [ ]:
####### Webcam and camera module useful for opening the webcam through google collab#####
def VideoCapture():
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);

      video = document.createElement('video');
      video.setAttribute('playsinline', '');

      div.appendChild(video);

      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;

      await video.play();

      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }

    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg', 0.8);
            pendingResolve(result);
            
        })
    }

    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
        

    }

  ''')
  display(js)

def byte2image(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)

def image2byte(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x



In [ ]:
### please only use the classifier  extracted from plib as utils (code already mentioned above)other's available online might be outdated or try with the one u have
labels_dict={1:'without_mask',0:'with_mask'}  ### should be changed according to your model
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
VideoCapture()
eval_js('create()')#Use camera 0

# We load the xml file
classifier = cv2.CascadeClassifier(utils.get_haarcascade_path('haarcascade_frontalface_default.xml'))

while True:
    byte = eval_js('capture()')
    frame = byte2image(byte)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Resize the image to speed up detection
    mini = cv2.resize(frame, (frame.shape[1] // size, frame.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = frame[y:y+h, x:x+w]   #### frame size of the detection
        resized=cv2.resize(face_img,(224,224))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,224,224,3))   ############### should be same as the input to conv2d layers other wise throws error
        reshaped = np.vstack([reshaped])
        result=face_mask.predict(reshaped)################## traine mask detector model loaded 
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[label],4)
        cv2.rectangle(frame,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(frame, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2) ### dont change anything here , works fine
        
    # Show the image
    eval_js('showimg("{}")'.format(image2byte(frame)))
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break

eval_js('create()').release()    ### have not figured out how to stop a cam when inactive after sometime , so it keep on untill u stop this cell from executing

# Close all started windows
cv2.destroyAllWindows()

MessageError: ignored

In [ ]:
### Ignore Below######################

import numpy as np
import cv2
import random
from PIL import Image
import time


cap = cv2.VideoCapture(-1)
print(cap.isOpened())
frames = []
i=0
j = 5
while(j>0):
    # Capture frame-by-frame
    time.sleep(1)
    (ret, frame) = cap.read()
    i+=1
    print(j)
    j-=1
#     frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
#     frame = Image.fromarray(frame)
    frames.append(frame)

  # exit if Escape is hit
    if cv2.waitKey(10) == 27:
      break



# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
for i in range(len(frames)):
  cv2_imshow(frames[i])
  cv2.waitkey(0)
  cv2.destroyAllWindows()
# frame_data = []
# for i in range(len(frames)):
#   frame_data.append(pre_process_data(frames[i]))
  
# for i in range(len(frame_data)):
#   frame_predict = model1.predict(frame_data[i])
#   print("The image belongs to the class: {} with a confidence of {} %".format(class_values[np.argmax(frame_predict)],(frame_predict[0,np.argmax(frame_predict)]/sum(sum(frame_predict)))*100))


size = 4
VideoCapture()
eval_js('create()')
face_cascade = cv2.CascadeClassifier(
    utils.get_haarcascade_path('haarcascade_frontalface_default.xml'))
while True:
    byte = eval_js('capture()'
    frame = byte2image(byte)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
      cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    eval_js('showimg("{}")'.format(image2byte(frame)),)
  
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break 